In [1]:
import PyPDF2
import pandas as pd
import numpy as np
import sys, os
import json
import re

# root = r"C:\Users\Victor\Documents\Cursos\Python\Automatizacion\PDF"
root = r"../PDF/"
path = os.path.join(root, "PDF")

# Funcion que obtiene los archivos con su ruta y se encarga de clasificarlos en arreglos
def ArchivosPDF(root, path):
    Archivos_Acuse_Recepcion = []
    Archivos_Opinion_Cumplimiento = []
    Archivos_Acuse_Presentacion = []
    Archivos_Constancia = []
    Archivos_Diot = []
    Archivos_no_clasificados = []
    
    for path, subdirs, files in os.walk(root):
        for name in files:
            # print(os.path.join(path, name))
            if os.path.join(name).endswith(".pdf"):
                #Archivos Acuse Recepcion
                if os.path.join(name).startswith("Acuse Recepcion"):
                    Archivos_Acuse_Recepcion.append(os.path.join(path, name))
                elif os.path.join(name).startswith("Opinion Cumplimiento"):
                    Archivos_Opinion_Cumplimiento.append(os.path.join(path, name))
                elif os.path.join(name).startswith("Acuse Presentacion"):
                    Archivos_Acuse_Presentacion.append(os.path.join(path, name))
                elif os.path.join(name).startswith("Constancia"):
                    Archivos_Constancia.append(os.path.join(path, name))
                elif os.path.join(name).startswith("Diot"):
                    Archivos_Diot.append(os.path.join(path, name))
                else: 
                    Archivos_no_clasificados.append(os.path.join(path, name))
                    
    return Archivos_Acuse_Recepcion, Archivos_Opinion_Cumplimiento, Archivos_Acuse_Presentacion, Archivos_Constancia,Archivos_Diot, Archivos_no_clasificados


# -> Papelines de Extraccion 

# Funcion de extraccion de datos para archivos PDF Acuse Recepcion de Pagos
def exportDataAcusesRecepcion(Archivos):
    
    array_docs = []
    for i in range(len(Archivos)):
        
        
        pdf = open(Archivos[i], 'rb')
        # Asigna el pdf en una variable
        reader = PyPDF2.PdfReader(pdf)
        # Asigno que pagina va a mostrar
        page = reader._get_page(0)

        texto = page.extract_text()

        arreglo = np.asarray(texto.split('\n'))
        arreglo = (texto.split('\n'))
        
        array_doc = []
        for e in range(len(arreglo)):
            array_base = arreglo[e].split(': ')
            # print(array_base)
            if len(array_base) == 2:
                array_doc.append(array_base)
        json_doc = dict(array_doc)
        # print(json_doc)
        
    array_docs.append(array_doc)

    # df = pd.DataFrame (array_docs, columns = ['data'])
    print(len(array_docs))
    
    data = json.dumps(array_docs)
    print(data)
    
        # df = pd.DataFrame(arreglo, columns= ['ColumnA'])
        
    
    # return df
    
    
#   Funcion de prueba
#   Con el objetivo de traer unicamente campos especificados
def exportDataAcusePresentacion(Archivos):
    array_docs = []
    for i in range(len(Archivos)):
        
        
        pdf = open(Archivos[i], 'rb')
        # Asigna el pdf en una variable
        reader = PyPDF2.PdfReader(pdf)
        # Asigno que pagina va a mostrar
        page = reader._get_page(0)

        texto = page.extract_text()

        
        # arreglo = np.asarray(texto.split('\n'))
        arreglo = (texto.split('\n'))
        
        print(texto.replace(" ", "<>"))
    
    return #print(i)
        
        
# Funcion de extraccion de datos para archivos PDF Opinion Cumplimiento
def exportDataOpinionCumplimietno(Archivos):
    
    for i in range(len(Archivos)):
        
        pdf = open(Archivos[i], 'rb')
        # Asigna el pdf en una variable
        reader = PyPDF2.PdfReader(pdf)
        # Asigno que pagina va a mostrar
        page = reader._get_page(0)

        texto = page.extract_text()

        print(texto.split('\n'))


# Varibales de PDF's'
# Se asigna el tipo de pdf a un variable

Archivos_Acuse_Recepcion = (ArchivosPDF(root, path)[0])         #--> Solo contiene 1 archivo
Archivos_Opinion_Cumplimiento = (ArchivosPDF(root, path)[1])    #--> Solo contiene 1 archivo
Archivos_Acuse_Presentacion = (ArchivosPDF(root, path)[2])      #--> °Contiene archivos Dummies y 1 original para pruebas
Archivos_Constancia = (ArchivosPDF(root, path)[3])              #--> Solo contiene 1 archivo
Archivos_Diot = (ArchivosPDF(root, path)[4])         #--> Solo contiene 1 archivo
Archivos_no_clasificados = (ArchivosPDF(root, path)[5])         #--> Solo contiene 1 archivo

# df = exportDataAcusesRecepcion(Archivos_Acuse_Presentacion)

# exportDataAcusePresentacion(Archivos_Acuse_Presentacion)

print(Archivos_Constancia)
print(Archivos_Diot)

['../PDF/Constancias_Presentacion_impuestos\\Constancia 07-03-23.pdf', '../PDF/Constancias_Presentacion_impuestos\\Constancia.pdf']
['../PDF/Constancias_Presentacion_impuestos\\Diot 01ENE23DIOTCUADRAS.pdf']
